<a href="https://colab.research.google.com/github/vivekb13/Auroville-Consulting/blob/main/Agri_app_result_F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install rasterio

In [ ]:
import geopandas as gpd
import rasterio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from rasterio.mask import mask
from shapely.geometry import box
from rasterio.features import geometry_mask
from rasterio.enums import Resampling
from rasterio.warp import reproject, calculate_default_transform

# -------------------------------
# Step 1: Load the GIS Layers
# -------------------------------
def load_shapefile(shapefile_path):
    return gpd.read_file(shapefile_path).to_crs(epsg=32644)  # Ensure CRS is EPSG:32644

def load_raster(raster_path):
    return rasterio.open(raster_path)

# File Paths
forest_shp_path =r"/content/drive/MyDrive/AgriPV_files/AgriPV_app_25/AGRI_PV_2025/India_Split/7/Forest_split_part_7.shp"  # Vector: Forest (shapefile)
water_shp_path = r"/content/drive/MyDrive/AgriPV_files/AgriPV_app_25/AGRI_PV_2025/India_Split/7/Water_split_part_7.shp"  # Vector: Water (shapefile)
cropland_raster_path = r"/content/drive/MyDrive/AgriPV_files/AgriPV_app_25/AGRI_PV_2025/India_Split/7/Crop_split_part_7.tif" # Raster: Cropland
slope_raster_path = r"/content/drive/MyDrive/AgriPV_files/AgriPV_app_25/AGRI_PV_2025/India_Split/7/Slope_Perc_split_part_7.tif"  # Raster: Slope percentage
ghi_raster_path = r"/content/drive/MyDrive/AgriPV_files/AgriPV_app_25/AGRI_PV_2025/India_Split/7/GHI_split_part_7.tif"  # Raster: GHI values
# roads_shp_path = r"/content/drive/MyDrive/Agripv_app/National_Highways/National_Highways.shp"
# power_lines_shp_path = r"/content/drive/MyDrive/Agripv_app/Power_lines/Power_lines.shp"

# Load Data
forest_area = load_shapefile(forest_shp_path)
water_area = load_shapefile(water_shp_path)
cropland_raster = load_raster(cropland_raster_path)
slope_raster = load_raster(slope_raster_path)
ghi_raster = load_raster(ghi_raster_path)
# roads_area = load_shapefile(roads_shp_path)
# power_lines_area = load_shapefile(power_lines_shp_path)

# -------------------------------
# Step 2: Reclassify Cropland Raster
# -------------------------------
def reclassify_cropland_raster(cropland_raster):
    cropland_data = cropland_raster.read(1)  # Read first band
    return np.where(cropland_data == 5, 1, 0)  # Convert 5 → 1 (cropland), else 0

cropland_binary = reclassify_cropland_raster(cropland_raster)

# ----------------------------------------------
# Step 3: Create Buffer Zone Around Forest/Water
# ----------------------------------------------
def create_buffer_area(forest_area, water_area, buffer_distance):
    combined_area = gpd.GeoDataFrame(pd.concat([forest_area, water_area], ignore_index=False), crs=forest_area.crs)
    return combined_area.geometry.buffer(buffer_distance)  # Apply buffer

# # Get user input for parameters
buffer_distance = float(input("Enter buffer distance (meters, e.g., 100): "))
max_slope_percent = float(input("Enter maximum slope percentage (e.g., 8): "))
ghi_threshold = float(input("Enter GHI threshold (kWh/m²/day, e.g., 4.5): "))

buffer_area = create_buffer_area(forest_area, water_area, buffer_distance)
# buffer_area = create_buffer_area(forest_area, water_area)

# ----------------------------------------------
# Step 4: Resample Rasters to Match Cropland
# ----------------------------------------------
def resample_raster(source_raster, reference_raster):
    """ Resamples a raster to match the reference raster """
    transform, width, height = calculate_default_transform(
        source_raster.crs, reference_raster.crs,
        reference_raster.width, reference_raster.height,
        *reference_raster.bounds)

    resampled_data = np.zeros((height, width), dtype=source_raster.read(1).dtype)

    reproject(
        source=source_raster.read(1),
        destination=resampled_data,
        src_transform=source_raster.transform,
        dst_transform=transform,
        src_crs=source_raster.crs,
        dst_crs=reference_raster.crs,
        resampling=Resampling.nearest)

    return resampled_data

slope_resampled = resample_raster(slope_raster, cropland_raster)
ghi_resampled = resample_raster(ghi_raster, cropland_raster)

# --------------------------------------------------------
# Step 5: Mask the Cropland Raster to Remove Buffer Zones
# --------------------------------------------------------
def mask_raster(raster_data, raster_transform, buffer_area):
    """ Mask raster with buffer zone """
    mask_array = geometry_mask(buffer_area, transform=raster_transform, invert=True, out_shape=raster_data.shape)
    raster_data[mask_array] = 0
    return raster_data

# def mask_raster(raster_data, raster_transform,buffer_area):
#     """ Mask raster with buffer zone """
#     mask_array = geometry_mask(buffer_area, transform=raster_transform, invert=False, out_shape=raster_data.shape)
#     raster_data[mask_array] = 0
#     return raster_data

cropland_masked = mask_raster(cropland_binary, cropland_raster.transform, buffer_area )
# cropland_masked = mask_raster(cropland_binary, cropland_raster.transform, buffer_area)

# -----------------------------------------------
# Step 6: Apply Slope and GHI Conditions
# -----------------------------------------------
def filter_suitable_areas(cropland_masked, slope_resampled, max_slope_percent, ghi_resampled, ghi_threshold):
# def filter_suitable_areas(cropland_masked):
    """ Filter cropland areas based on slope and GHI criteria """

    # Mask out invalid slope values (-9999) if present
    slope_resampled = np.where(slope_resampled < -1000, 0, slope_resampled)

    # 1. Apply slope condition (<= max_slope_percent)
    slope_within_limit = slope_resampled <= max_slope_percent

    # 2. Apply GHI condition (>= ghi_threshold)
    ghi_suitable = ghi_resampled >= ghi_threshold

    # 3. Combine all conditions
    # suitable_area = (cropland_masked == 1)
    suitable_area = (cropland_masked == 1) & slope_within_limit & ghi_suitable

    # 4. Calculate total area (each pixel represents 1000m x 1000m = 1 km² = 247.1 acres)
    pixel_size_km2 = 1  # Since resolution is 1000m x 1000m
    total_suitable_area_acres = np.sum(suitable_area) * pixel_size_km2 * 247.1

    return suitable_area, total_suitable_area_acres

# suitable_land, total_area = filter_suitable_areas(cropland_binary)
suitable_land, total_area = filter_suitable_areas(cropland_masked, slope_resampled, max_slope_percent, ghi_resampled, ghi_threshold)
# suitable_land, total_area = filter_suitable_areas(cropland_masked)

# -----------------------------------------------
# Step 7: Display Results
# -----------------------------------------------
if total_area > 0:
    print(f"Total Available Area for Agri-PV: {total_area:.2f} acres")

    # Plot Suitable Area
    plt.imshow(suitable_land, cmap="Oranges")
    plt.title("Suitable Land for Agri-PV (Filtered)")
    plt.colorbar()
    plt.show()
else:
    print("No suitable land found based on the given conditions.")


# Define output file path
output_raster_path = r"/content/drive/MyDrive/AgriPV_files/AgriPV_app_25/AGRI_PV_2025/Results/T_L7.tif"

# Convert suitable_land to binary (ensure only 1 values are kept)
suitable_land_binary = (suitable_land == 1).astype(rasterio.uint8)  # Convert to 0 and 1

# Save the suitable land as a raster file
with rasterio.open(
    output_raster_path,
    'w',
    driver='GTiff',
    height=suitable_land_binary.shape[0],
    width=suitable_land_binary.shape[1],
    count=1,
    dtype=rasterio.uint8,  # Use uint8 for binary data
    crs=cropland_raster.crs,
    transform=cropland_raster.transform
) as dst:
    dst.write(suitable_land_binary, 1)  # Write to band 1

print(f"Suitable land saved to {output_raster_path}")



#Commercial Feasilibilty

In [ ]:
import geopandas as gpd
import rasterio
import numpy as np
from shapely.geometry import box
from rasterio.mask import mask
from rasterio.transform import from_origin
import matplotlib.pyplot as plt

# Step 1: Read the raster file
raster_path = r"/content/drive/MyDrive/AgriPV_files/AgriPV_app_25/AGRI_PV_2025/Results/T_L7.tif"
with rasterio.open(raster_path) as src:
    raster = src.read(1)  # Read the first band
    transform = src.transform  # Get the transformation matrix
    crs = src.crs  # Get the coordinate reference system (CRS)

# Step 2: Read the shapefiles (roads and powerlines)
# Step 2: Read the shapefiles (roads and powerlines)
roads_path = r"/content/drive/MyDrive/AgriPV_files/AgriPV_app_25/AGRI_PV_2025/India_Split/7/Roads_NH_SH_split_part_7.shp"
powerlines_path = r"/content/drive/MyDrive/AgriPV_files/AgriPV_app_25/AGRI_PV_2025/India_Split/7/Powerlines_split_part_7.shp"


roads = gpd.read_file(roads_path)
powerlines = gpd.read_file(powerlines_path)

# Ensure the shapefiles have the same CRS as the raster
roads = roads.to_crs(crs)
powerlines = powerlines.to_crs(crs)

# Step 3: Create different buffers around the shapefiles
# Set different buffer distances for roads and powerlines (in map units, e.g., meters)
roads_buffer_distance = float(input("Enter Road buffer distance (meters, e.g., 100): "))
powerlines_buffer_distance = float(input("Enter Powerlines buffer distance (meters, e.g., 100): "))

roads_buffer = roads.buffer(roads_buffer_distance)
powerlines_buffer = powerlines.buffer(powerlines_buffer_distance)

# combined_buffer = gpd.GeoSeries(pd.concat([roads_buffer, powerlines_buffer])).unary_union
combined_buffer = gpd.GeoSeries(pd.concat([roads_buffer, powerlines_buffer])).union_all()



# Step 5: Overlay the combined buffer on the raster
geom = [combined_buffer]  # Convert to a list for the mask function
with rasterio.open(raster_path) as src:
    out_image, out_transform = mask(src, geom, crop=True)
    out_image = out_image[0]  # If the raster has multiple bands, select the first one

# Step 6: Calculate the area of the pixels that intersect the buffer (including partially intersecting)
# Calculate pixel area (depends on the raster's resolution)
pixel_area = abs(out_transform[0] * out_transform[4])  # resolution * resolution

# Step 6.1: Identify the pixels inside the buffer (out_image > 0 means they intersect with the buffer)
buffer_intersect_pixels = (out_image > 0)  # Create a mask where the pixels are inside or intersecting the buffer

# Step 6.2: Calculate the total area of intersecting pixels
intersecting_area_in_pixels = np.sum(buffer_intersect_pixels) * pixel_area

# Convert area to acres (1 acre = 43560 square feet)
intersecting_area_in_acres = intersecting_area_in_pixels / 43560

print(f"Total area of pixels that intersect with the combined buffer (in acres): {intersecting_area_in_acres}")

# # Step 7: Save the output
# # Save the combined buffer to a shapefile
# output_buffer_path = "combined_buffer.shp"
# gpd.GeoSeries([combined_buffer]).to_file(output_buffer_path)

# Save the rasterized output as a new raster file
output_raster_path = r"/content/drive/MyDrive/AgriPV_files/AgriPV_app_25/AGRI_PV_2025/Results/L7.tif"
with rasterio.open(
    output_raster_path,
    'w',
    driver='GTiff',
    height=out_image.shape[0],
    width=out_image.shape[1],
    count=1,
    dtype=out_image.dtype,
    crs=crs,
    transform=out_transform
) as dst:
    dst.write(out_image, 1)

# Optional: Plot the result
plt.imshow(out_image, cmap='gray')
plt.title("Overlay of Combined Buffer on Raster")
plt.show()


In [ ]:
# import geopandas as gpd
# import rasterio
# import numpy as np
# from shapely.geometry import box
# from rasterio.mask import mask
# from rasterio.transform import from_origin
# import matplotlib.pyplot as plt

# # Step 1: Read the raster file
# raster_path = r"/content/drive/MyDrive/AGRI_PV_2023/res2.tif"
# with rasterio.open(raster_path) as src:
#     raster = src.read(1)  # Read the first band
#     transform = src.transform  # Get the transformation matrix
#     crs = src.crs  # Get the coordinate reference system (CRS)

# # Step 2: Read the shapefiles (roads and powerlines)
# # Step 2: Read the shapefiles (roads and powerlines)
# roads_path = r"/content/drive/MyDrive/Agripv_app/National_Highways/National_Highways.shp"
# powerlines_path = r"/content/drive/MyDrive/Agripv_app/Power_lines/Power_lines.shp"


# roads = gpd.read_file(roads_path)
# powerlines = gpd.read_file(powerlines_path)

# # Ensure the shapefiles have the same CRS as the raster
# roads = roads.to_crs(crs)
# powerlines = powerlines.to_crs(crs)

# # Step 3: Create different buffers around the shapefiles
# # Set different buffer distances for roads and powerlines (in map units, e.g., meters)
# roads_buffer_distance = 400  # Buffer distance for roads
# powerlines_buffer_distance = 500  # Buffer distance for powerlines

# roads_buffer = roads.buffer(roads_buffer_distance)
# powerlines_buffer = powerlines.buffer(powerlines_buffer_distance)

# # combined_buffer = gpd.GeoSeries(pd.concat([roads_buffer, powerlines_buffer])).unary_union
# combined_buffer = gpd.GeoSeries(pd.concat([roads_buffer, powerlines_buffer])).union_all()



# # Step 5: Overlay the combined buffer on the raster
# geom = [combined_buffer]  # Convert to a list for the mask function
# with rasterio.open(raster_path) as src:
#     out_image, out_transform = mask(src, geom, crop=True)
#     out_image = out_image[0]  # If the raster has multiple bands, select the first one

# # Step 6: Calculate the area of the pixels that intersect the buffer (including partially intersecting)
# # Calculate pixel area (depends on the raster's resolution)
# pixel_area = abs(out_transform[0] * out_transform[4])  # resolution * resolution

# # Step 6.1: Identify the pixels inside the buffer (out_image > 0 means they intersect with the buffer)
# buffer_intersect_pixels = (out_image > 0)  # Create a mask where the pixels are inside or intersecting the buffer

# # Step 6.2: Calculate the total area of intersecting pixels
# intersecting_area_in_pixels = np.sum(buffer_intersect_pixels) * pixel_area

# # Convert area to acres (1 acre = 43560 square feet)
# intersecting_area_in_acres = intersecting_area_in_pixels / 43560

# print(f"Total area of pixels that intersect with the combined buffer (in acres): {intersecting_area_in_acres}")

# # # Step 7: Save the output
# # # Save the combined buffer to a shapefile
# # output_buffer_path = "combined_buffer.shp"
# # gpd.GeoSeries([combined_buffer]).to_file(output_buffer_path)

# # Save the rasterized output as a new raster file
# output_raster_path = r"/content/drive/MyDrive/AGRI_PV_2023/rasterized_buffer.tif"
# with rasterio.open(
#     output_raster_path,
#     'w',
#     driver='GTiff',
#     height=out_image.shape[0],
#     width=out_image.shape[1],
#     count=1,
#     dtype=out_image.dtype,
#     crs=crs,
#     transform=out_transform
# ) as dst:
#     dst.write(out_image, 1)

# # Optional: Plot the result
# plt.imshow(out_image, cmap='gray')
# plt.title("Overlay of Combined Buffer on Raster")
# # plt.show()
